# WB

In [ ]:
import gc
import test_tools
import numpy as np
import pandas as pd
import geopandas as gpd
from procesa_bases import load_WB_country_data

PATH = "D:\World Bank\CLIENT v2"
DATA_RAW = rf"{PATH}\Data\Data_raw"
DATA_PROC = rf"{PATH}\Data\Data_proc"
DATA_OUT = rf"{PATH}\Data\Data_out"
GPW_PATH = rf"D:\Datasets\Gridded Population of the World"



## Genera mapa con etiquetas de zona (adm0 adm1 adm2)

In [ ]:
# Load ID dataset
gdf = gpd.read_feather(r"D:\World Bank\CLIENT v2\Data\Data_proc\WB_country_IDs.feather")
gdf.columns = gdf.columns.str.lower()
gdf = gdf.rename(columns={"id":"ID"}).drop(columns="objectid")

# Add names from the original WB adm2 dataset
gdf_raw = load_WB_country_data()
gdf_raw.columns = gdf_raw.columns.str.lower()
gdf_raw = gdf_raw[["adm0_code", "adm1_code", "adm2_code", "adm0_name", "adm1_name", "adm2_name", "geometry"]]
assert gdf_raw.duplicated(subset=["adm0_code", "adm1_code", "adm2_code"]).sum() == 0, "There are duplicated entries in the raw dataset!!"

# Merge both datasets to assert that the codes are correct and consistent
gdf = gdf.merge(gdf_raw.drop(columns="geometry"), how="outer", on=["adm0_code", "adm1_code", "adm2_code"], indicator=True, validate="1:1")
assert (gdf._merge == "both").all(), "There are problems with the merge!!"
gdf = gdf.drop(columns="_merge")

gdf.drop(columns="ID").to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_map.csv", index=False) # Export without the ID column

In [43]:
# Set admin level to categorical dtype (when the dataset is expanded, it will be more memory efficient)
gdf["ID"]        = gdf["ID"].astype("category")
gdf["adm0_code"] = gdf["adm0_code"].astype("category")
gdf["adm1_code"] = gdf["adm1_code"].astype("category")
gdf["adm2_code"] = gdf["adm2_code"].astype("category")
gdf = gdf.set_index("ID")

gdf = gdf.drop(columns=["adm0_name","adm1_name","adm2_name"])

## Genera datos de shocks climáticos

In [5]:
gdf_raw = gpd.read_file(r"D:\World Bank\CLIENT v2\Data\Data_raw\world_bank_adm2\world_bank_adm2.shp")
gdf_raw.columns = gdf_raw.columns.str.lower()
gdf_raw = gdf_raw[["adm0_code", "adm1_code", "adm2_code", "adm0_name", "adm1_name", "adm2_name"]]

In [44]:
import pandas as pd
import itertools


def expand_dataset(df, gdf):
                    
    # Collect all dimension values from df
    all_years      = df.index.get_level_values("year").categories
    all_variables  = df.index.get_level_values("variable").categories
    all_thresholds = df.index.get_level_values("threshold").categories
    all_measures   = df.index.get_level_values("measure").categories
    all_regions    = gdf.index.categories # ID is the index of gdf

    # Convert each list to a small DataFrame
    df_years      = pd.DataFrame({'year': all_years}, dtype='category')
    df_variables  = pd.DataFrame({'variable': all_variables}, dtype='category')
    df_thresholds = pd.DataFrame({'threshold': all_thresholds}, dtype='category')
    df_measures   = pd.DataFrame({'measure': all_measures}, dtype='category')
    df_regions    = pd.DataFrame({'ID': all_regions}, dtype='category')

    # Step-by-step merges using how='cross'
    df_temp = df_years.merge(df_variables, how='cross')
    df_temp = df_temp.merge(df_regions, how='cross')
    df_temp = df_temp.merge(df_thresholds, how='cross')
    df_temp = df_temp.merge(df_measures, how='cross')
    expanded_without_data = df_temp.set_index(["ID", "year", "variable", "threshold", "measure"])
    
    # add admcodes to the expanded set
    expanded_without_data = expanded_without_data.join(
        gdf.drop(columns=["geometry"]),
        how="left",
        on="ID",
        validate="m:1"
    )
    
    # Merge original data (df) onto the expanded set
    expanded_with_data = expanded_without_data.join(
        df,
        how="left",
        validate="1:1",
        rsuffix="_y"
    ).reset_index().drop(columns="ID")
    
    expanded_with_data = test_tools.assert_correct_admcodes(expanded_with_data)        

    return expanded_with_data

In [ ]:
from importlib import reload

## Generados Nico
# Set dtypes to make this loading efficient
dtypes = {"year": np.int16, "variable":"category", "threshold":"category", "area_affected":np.float32, "population_affected":np.float32, "ID":np.int64}# "adm2_code": np.int16, "adm1_code": np.int16, "adm0_code": np.int16,

for shock in ["floods", "drought", "hurricanes", "intenserain", "heatwaves", "coldwaves"]:
    print(shock)
    df = pd.read_csv(
        rf"D:\World Bank\CLIENT v2\Data\Data_out\WB_{shock}_long.csv",
        dtype=dtypes, 
        usecols=dtypes.keys(),
    )
        
    # Set ID to categorical dtype (this is after loading as int to match with the categories of gdf)
    df["ID"] = df["ID"].astype("category")
    
    # Reshape to long format
    df = df.melt(id_vars=["ID", "year", "variable", "threshold"], var_name="measure", value_name="value")

    # Set categorical and index to make faster merges
    df["measure"] = df["measure"].astype("category")
    df["year"] = df["year"].astype("category")
    df = df.set_index(["ID"])    
    
    # Add adm0, adm1 and adm2 codes    
    df = gdf.drop(columns=["geometry"]).join(df, on=["ID"], how="inner", validate="1:m")
    df = df.reset_index()

    # Set index to make faster merges and expand dataset
    #   Replace columns with null categories with zeros before setting the index to make it work as expected:
    index = ["ID", "year", "variable", "threshold", "measure"]
    for col in index:
        if (df[col].dtype == "category"):
            if (df[col].cat.categories.shape[0]==0):
                df[col] = df[col].astype(float).fillna(0)
                df[col] = df[col].astype("category")

    df = df.set_index(index)
    df = expand_dataset(df, gdf)

    # Test the output
    test_tools.assert_correct_colnames(df)
    test_tools.assert_correct_shape(df, gdf)

    # Export
    df.to_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_{shock}.csv", index=False)

    df = None
    gc.collect()    

In [ ]:
# test all is ok
from importlib import reload
reload(test_tools)
gdf = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_map.csv")
# gdf = gpd.GeoDataFrame(gdf, geometry=gpd.GeoSeries.from_wkt(gdf["geometry"]))

for shock in ["floods", "drought", "hurricanes", "intenserain", "heatwaves", "coldwaves"]:
    print("Verifying", shock)
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_{shock}.csv")
    test_tools.validate_climate_dataset(df, gdf)


In [ ]:
gdf = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\WB_map.csv")
gdf

In [98]:
test_b = gdf.sort_values(["adm0_code", "adm1_code", "adm2_code"])
test_b = test_b.set_index(["adm0_code", "adm1_code", "adm2_code"])
test_b = test_b.reset_index()

In [99]:
test_a = df.query("year==2000 and measure=='area_affected'").sort_values(["adm0_code", "adm1_code", "adm2_code"])
test_a = test_a.set_index(["adm0_code", "adm1_code", "adm2_code"])
test_a = test_a.reset_index()

In [ ]:
test_a.merge(test_b, on=["adm0_code", "adm1_code", "adm2_code"], how="outer", validate="1:1", indicator=True)._merge.value_counts()

In [ ]:
gdf

In [ ]:
gdf

In [ ]:
df

# IPUMS

In [6]:
import pandas as pd
import geopandas as gpd

gdf_full = gpd.read_feather(r"D:\World Bank\CLIENT v2\Data\Data_proc\IPUMS_country_IDs.feather")
gdf_full = gdf_full.drop(columns=["ID"])
gdf_full = gdf_full.rename(columns={"CNTRY_CODE":"adm0", "GEOLEVEL1":"adm1", "GEOLEVEL2":"adm2"})
gdf_full[["adm0_name", "adm1_name", "adm2_name"]] = "To be filled"
ids = ["adm0", "adm2"]


## Nacional

In [ ]:
import os

# Shocks
path = r"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete"
       
files = os.listdir(path)
files = [f for f in files if "HC_national_data" in f and f.endswith(".csv")]

dfs = []
for file in files:
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete\{file}")
    df["s3"] = pd.NA
    df["s4"] = pd.NA
    
    s3cols = ["s3a", "s3b", "s3c", "s3d", "s3f"]
    s4cols = ["s4a", "s4b", "s4c"]

    for col in s3cols: 
        df["s3"] = df["s3"].fillna(df[col])
        assert (df[s3cols].notna().sum(axis=1) <= 1).all(), f"{df[(df[s3cols].notna().sum(axis=1) > 1)]}"
    for col in s4cols:
        df["s4"] = df["s4"].fillna(df[col])
        assert (df[s4cols].notna().sum(axis=1) <= 1).all(), f"{df[(df[s3cols].notna().sum(axis=1) > 1)]}"

    dfs += [df]
    
df = pd.concat(dfs)
for col in df.columns:
    assert df[col].isna().all() == False

# Drop s3* columns
df = df.drop(columns=[col for col in df.columns if ("s3" in col or "s4" in col) and (col != "s3" and col != "s4")])
# Order variables
df = df[["adm0", "s1", "s2", "s3", "s4", "s5", "outcome", "new", "v"]]
df = df.rename(columns={"new":"time", "v": "value", "status":"treatment"})
df.loc[df.s1 == "Hurricane", "s5"] = df.loc[df.s1 == "Hurricane", "s5"] / 100

df = df.merge(gdf_full[["adm0"]].drop_duplicates(), on=["adm0"], validate="m:1")
print(f"Hay datos de {df.adm0.unique().size} países")
df.to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_national_data.csv", index=False)

In [8]:
labels = {
    "s1":"Shock",
    "s2":"Weight",
    "s3": {
        "Cold wave":"Temperature <0 °C",
        "Heat wave":"Degrees (°C)",
        "Drought":"Drought indicator",
        "Intense rain":"Number of days",
        "Hurricane":"Category"
    },
    "s4": {
        "Cold wave":"Standard Deviations from historical mean",
        "Heat wave":"Standard Deviations from historical mean",
        "Drought":"Standard Deviations from historical mean",
        "Intense rain":"Rainfall (mm)",
        "Hurricane":"Distance from center of the storm (degrees)" # Fixme: turn to km
    },
    "s5": r"Threshold (% affected)",
}

## ADM2

In [ ]:
import os
from tqdm import tqdm 

# Shocks
path = r"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete"
       
files = os.listdir(path)
files = [f for f in files if "HC_geodata" in f and f.endswith(".csv")]

dfs = []
for file in tqdm(files):
    df = pd.read_csv(rf"D:\World Bank\CLIENT v2\Data\Data_out\HC Treatment Complete\{file}")
    df["s3"] = pd.NA
    df["s4"] = pd.NA
    
    s3cols = ["s3a", "s3b", "s3c", "s3d", "s3f"]
    s4cols = ["s4a", "s4b", "s4c"]

    for col in s3cols: 
        df["s3"] = df["s3"].fillna(df[col])
        assert (df[s3cols].notna().sum(axis=1) <= 1).all(), f"{df[(df[s3cols].notna().sum(axis=1) > 1)]}"
    for col in s4cols:
        df["s4"] = df["s4"].fillna(df[col])
        assert (df[s4cols].notna().sum(axis=1) <= 1).all(), f"{df[(df[s3cols].notna().sum(axis=1) > 1)]}"

    dfs += [df]
    
df = pd.concat(dfs)
for col in df.columns:
    assert df[col].isna().all() == False

# Drop s3* columns
df = df.drop(columns=[col for col in df.columns if ("s3" in col or "s4" in col) and (col != "s3" and col != "s4")])
# Order variables
df = df[["adm0", "adm1", "adm2", "s1", "s2", "s3", "s4", "s5", "outcome", "status", "diftime"]]
df = df.rename(columns={"status":"treatment_sub", "diftime":"diff"})
df.loc[df.s1 == "Hurricane", "s5"] = df.loc[df.s1 == "Hurricane", "s5"] / 100

df.merge(gdf_full[["adm0", "adm2"]], on=["adm0", "adm2"], validate="m:1", how="inner")
print(f"Hay datos de {df.adm0.unique().size} países")

df.to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_data.csv", index=False)

## Map

In [ ]:
df = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_data.csv", dtype={"adm0":"float", "adm2":"float"})
df = df[["adm0", "adm2"]].drop_duplicates()
gdf_full.merge(df, on=["adm0", "adm2"], validate="1:m", how="inner").to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_map.csv", index=False)

In [ ]:
df_map = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_map.csv")
df_national = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_national_data.csv")
df_geo = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_data.csv")

In [12]:
m1 = df_map.merge(df_national.drop_duplicates(subset="adm0"), on=["adm0"], validate="m:1", how="outer", indicator=True)
assert m1[m1._merge != "both"].shape[0] == 0
m2 = df_map.merge(df_geo, on=["adm0", "adm2"], validate="1:m", how="outer", indicator=True)
assert m2[m2._merge != "both"].shape[0] == 0

## Export Labels

In [13]:
df = pd.read_excel(r"D:\World Bank\CLIENT v2\Data\Data_raw\button_labels.xlsx")

In [14]:
df.to_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\selector_labels.csv", index=False)

## Check consistency

In [ ]:
from shapely import wkt

# Assert that all data merges correctly

gdf = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\IPUMS_map.csv")
gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(gdf, crs='epsg:4326')

df_adm2 = pd.read_csv(r"D:\World Bank\CLIENT v2\Data\Data_out\for webpage\HC_geo_data.csv")

In [ ]:
gdf.merge(df_adm2, on=["adm0", "adm2"], validate="1:m", how="outer", indicator=True)._merge.value_counts()

In [ ]:
merged = gdf_full[["CNTRY_CODE", "GEOLEVEL2", "geometry"]].merge(df, right_on=["adm0", "adm2"], left_on=["CNTRY_CODE", "GEOLEVEL2"], how="outer", indicator=True)
merged = merged[~merged.CNTRY_CODE.isin([231,276,356,368,376,504,566,586,662])]
pd.crosstab(merged[merged._merge!="both"].CNTRY_CODE, merged[merged._merge!="both"]._merge)

In [ ]:
# Palestina no está porque solo tiene after. El resto 10/10

import folium
m = merged[merged._merge!="both"].drop_duplicates(subset=["CNTRY_CODE", "GEOLEVEL2"]).explore()

# add control for layers
folium.LayerControl().add_to(m)

m